In [1]:
! pip install dalex pymysql --user >> /dev/null

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [16]:
pip install --upgrade pip'

/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import time
import random
import os, requests, json
import datetime
import joblib
from sqlalchemy import create_engine


In [3]:
test_df = pd.read_csv("NoShowPrediction.csv")
input_features = ["Employed", "Days Since Last Scheduled Appointment", "Education Level_PHD", 
                       "Major Depression", "Hospital Admssions", "Chronic pain", "Charlson Index", 
                       "Income per year", "Education Level_Masters", "Diabetes", "Drug Dependence", 
                       "Scheduled within 14 days", "Cost Covered by VA", "Winter", 
                       "Education Level_High School", "Distance to VA", "No. of previously scheduled visits", 
                       "Cardiac Condition", "Total Number of No-Shows", "Marital Status", "Age",
                    ]
# test_df = test_df[input_features]

In [4]:
### SQL details
hostname = "10.142.0.3"
databasename = "monitoring"
username = "root"
password = "test"
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                            .format(host=hostname, db=databasename, user=username, pw=password))
try:
    if engine.connect():
        print("Connection to SQL is successful")
except Exception as e:
    print(e)

Connection to SQL is successful


In [5]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
predict_url = "https://10.142.0.3:32222/dkube/inference/abhijeetc/feb271b6fd:predict"

In [6]:
no_of_samples = random.randint(10,15)
print(no_of_samples)

10


In [7]:
sample_df = test_df.sample(no_of_samples)
sample_df

,Appointment ID,Employed,Days Since Last Scheduled Appointment,Education Level_PHD,Major Depression,Hospital Admssions,Chronic pain,Charlson Index,Income per year,Education Level_Masters,...,Winter,Education Level_High School,Distance to VA,No. of previously scheduled visits,Cardiac Condition,Total Number of No-Shows,Marital Status,Age,Appointment Date Time,No-show %
689,MSL24409,1,26,0,0,0,0,0,476000,1,...,0,0,55,5,0,1,1,62,2020-01-26 19:30,0.466241
936,MSL24888,1,10,0,1,0,1,0,157000,0,...,1,0,0,2,1,1,0,30,2020-02-17 16:15,0.371108
116,MSL23158,0,28,0,0,0,0,0,19000,1,...,0,0,72,14,0,1,1,74,2020-01-05 14:30,0.124126
969,MSL24933,1,38,0,1,0,1,0,354000,0,...,0,0,2,1,1,0,0,59,2020-02-24 11:15,0.011285
691,MSL24412,0,60,0,0,0,0,0,10000,1,...,0,0,14,7,0,1,1,76,2020-01-27 10:00,0.144179
240,MSL23444,0,40,0,0,0,0,0,14000,1,...,0,0,77,9,0,1,1,98,2020-01-09 19:45,0.112258
138,MSL23207,0,49,0,0,0,0,10,17000,1,...,0,0,16,12,0,1,1,99,2020-01-06 11:15,0.067808
904,MSL24829,1,3,0,1,0,1,0,356000,0,...,1,0,4,2,1,1,0,39,2020-02-10 19:30,0.313176
42,MSL22988,0,24,0,0,0,0,0,17000,1,...,0,0,70,10,0,1,1,89,2020-01-02 19:00,0.121326
889,MSL24802,0,35,0,1,0,1,0,13000,0,...,0,0,5,2,1,1,0,27,2020-02-10 10:45,0.324106


In [8]:
values = sample_df[input_features].values
print(values)

[[     1     26      0      0      0      0      0 476000      1      0
       0      0     14      0      0     55      5      0      1      1
      62]
 [     1     10      0      1      0      1      0 157000      0      1
       1      1      9      1      0      0      2      1      1      0
      30]
 [     0     28      0      0      0      0      0  19000      1      0
       0      0     10      0      0     72     14      0      1      1
      74]
 [     1     38      0      1      0      1      0 354000      0      1
       1      0      0      0      0      2      1      1      0      0
      59]
 [     0     60      0      0      0      0      0  10000      1      0
       0      0     51      0      0     14      7      0      1      1
      76]
 [     0     40      0      0      0      0      0  14000      1      0
       0      0     15      0      0     77      9      0      1      1
      98]
 [     0     49      0      0      0      0     10  17000      1      0
    

In [ ]:
while True:
    no_of_samples = random.randint(10,15)
    sample_df = test_df.sample(no_of_samples)
    values = sample_df[input_features].values
    payload = {"instances": values.tolist(), "token": token}
    r = requests.post(predict_url, json=payload, 
                      headers = {'authorization': "Bearer " + token}, 
                      verify = False)
    if r.status_code == 200:
        start = datetime.datetime.utcnow()
        end = start + datetime.timedelta(seconds=10)
        predictions = r.json()["predictions"]
        labels = sample_df["No-show %"]
        timestamps = pd.date_range(start, end, len(predictions))
        labelled_df = pd.DataFrame({
            "AppointmentDateTime": timestamps,
            "No-show %": predictions,
            "No-show-label": labels
        })
        labelled_df.to_sql("no_show_label", engine, if_exists="append", index=False)
        print("labelled data pushed")
    time.sleep(60)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.142.0.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


labelled data pushed


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.142.0.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


labelled data pushed


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.142.0.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


...

labelled data pushed


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.142.0.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


labelled data pushed


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.142.0.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


labelled data pushed


## query
select * from no_show_label

## Ground truth col name
No-show-label

## timestamp col name
AppointmentDateTime